In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, HBox, VBox, Output, ToggleButton
import statsmodels.api as sm
import ipywidgets as widgets

# Initial parameters
alpha_slider = FloatSlider(value=5.0, min=-2, max=7, step=0.05, description=r'Alpha:')
beta_slider = FloatSlider(value=.7, min=-2, max=3, step=0.05, description=r'Beta:')
variance_slider = FloatSlider(value=5.0, min=.2, max=25.0, step=0.1, description=r'Variance of Error:')
mean_epsilon_slider = FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1, description=r'Mean of $\varepsilon$:')

# Buttons
add_button = Button(description='Add 10 samples')
subtract_button = Button(description='Subtract 10 samples')
reset_button = Button(description='Reset samples')

# Toggle button for intercept
intercept_toggle = ToggleButton(value=True, description='Include intercept')

# Output widget for plot
output = Output()

# Variables to hold sample data
X_samples = np.array([]).reshape(-1, 1)
Y_samples = np.array([])

# Function to update the plot
def update_plot(*args):
    with output:
        output.clear_output(wait=True)
        if len(X_samples) == 0:
            print("No samples to display.")
            return

        alpha = alpha_slider.value
        beta = beta_slider.value
        variance = variance_slider.value
        mean_epsilon = mean_epsilon_slider.value
        include_intercept = intercept_toggle.value

        # Prepare X data for statsmodels
        if include_intercept:
            X = sm.add_constant(X_samples)  # Adds a column of ones for the intercept
        else:
            X = X_samples  # No intercept term

        # Fit the model using statsmodels
        model = sm.OLS(Y_samples, X).fit()

        # Get estimated parameters
        if include_intercept:
            alpha_hat = model.params[0]
            beta_hat = model.params[1]
        else:
            alpha_hat = 0.0
            beta_hat = model.params[0]

        # Plot data and regression line
        plt.figure(figsize=(8,6))
        plt.scatter(X_samples, Y_samples, color='blue', label='Sample data')
        plt.axhline(y=0, linestyle='--', color='black', alpha=0.5) 
        plt.axvline(x=0, linestyle='--', color='black', alpha=0.5) 

        # Plot true regression line
        X_line = np.linspace(np.min(X_samples), np.max(X_samples), 100).reshape(-1,1)
        Y_true = alpha + beta * X_line + mean_epsilon
        plt.plot(X_line, Y_true, color='green', linestyle='--', label='True model')

        # Plot fitted regression line
        if include_intercept:
            X_line_with_const = sm.add_constant(X_line)
            Y_pred = model.predict(X_line_with_const)
        else:
            Y_pred = model.predict(X_line)
        plt.plot(X_line, Y_pred, color='red', label='Fitted model')

        # Set title with estimated parameters
        title_text = f"Estimated parameters: "
        if include_intercept:
            title_text += r"$\hat{{\alpha}}$ = {:.2f}, ".format(alpha_hat)
        else:
            title_text += r"$\hat{{\alpha}}$ = 0.0 (fixed), "
        title_text += r"$\hat{{\beta}}$ = {:.2f}".format(beta_hat)
        plt.title(title_text)
        plt.xlabel(f'X (N Observations: {len(Y_samples):.0f})')
        plt.ylabel('Y')
        plt.legend()
        plt.show()

# Functions for button clicks
def add_samples(b):
    global X_samples, Y_samples
    alpha = alpha_slider.value
    beta = beta_slider.value
    variance = variance_slider.value
    mean_epsilon = mean_epsilon_slider.value

    # Generate 10 new samples
    X_new = np.random.uniform(0, 10, 10).reshape(-1, 1)
    epsilon = np.random.normal(mean_epsilon, np.sqrt(variance), 10)
    Y_new = alpha + beta * X_new.flatten() + epsilon

    # Append to existing samples
    X_samples = np.vstack([X_samples, X_new])
    Y_samples = np.concatenate([Y_samples, Y_new])

    update_plot()

def subtract_samples(b):
    global X_samples, Y_samples
    # Remove last 10 samples
    if len(X_samples) >= 10:
        X_samples = X_samples[:-10]
        Y_samples = Y_samples[:-10]
    else:
        X_samples = np.array([]).reshape(-1, 1)
        Y_samples = np.array([])

    update_plot()

def reset_samples(b):
    global X_samples, Y_samples
    X_samples = np.array([]).reshape(-1, 1)
    Y_samples = np.array([])
    update_plot()

# Attach button click events
add_button.on_click(add_samples)
subtract_button.on_click(subtract_samples)
reset_button.on_click(reset_samples)

# Update plot when parameters change
alpha_slider.observe(update_plot, 'value')
beta_slider.observe(update_plot, 'value')
variance_slider.observe(update_plot, 'value')
mean_epsilon_slider.observe(update_plot, 'value')
intercept_toggle.observe(update_plot, 'value')

# Layout
controls = VBox([
    HBox([alpha_slider, beta_slider, variance_slider]),
    HBox([add_button, subtract_button, reset_button, intercept_toggle])
])

display(VBox([controls, output]))

# Initial plot
update_plot()